# Algorithms Semester Project

Below, I will be conducting an analysis of Gemini 2.5's proficiency of generating code for the following algorithms:

- BFS
- DFS
- Kruskal’s
- Prim’s
- Dijkstra’s
- Bellman-Ford

Below are necessary import statements required for our algoritms:

In [304]:
import time
import heapq # For the priority queue (min-heap)
import collections
import sys

## BFS and DFS

Below is the graph I will use to test the algorithms:

In [305]:
graph_bfs_dfs = {
0: [1, 2], 
1: [2], 
2: [0, 3], 
3: [3]
}

### BFS (breadth first search)

In [306]:
def bfs(graph, start_node):
  start_time = time.perf_counter()
  """
  Performs Breadth-First Search on a graph.

  Args:
    graph: A dictionary representing the adjacency list of the graph.
           Example: {'A': ['B', 'C'], 'B': ['D', 'E'], 'C': ['F'], ...}
    start_node: The node from which to start the traversal.

  Returns:
    A list containing the nodes in the order they were visited,
    or None if the start_node is not in the graph. Returns an empty
    list if the graph is empty.
  """

  if not graph:
      print("Graph is empty.")
      return [] # Return empty list for empty graph

  if start_node not in graph:
      print(f"Start node '{start_node}' not found in the graph.")
      return None # Indicate start node wasn't found

  # Keep track of visited nodes to avoid cycles
  visited = set()

  # Use a deque (double-ended queue) for efficient queue operations (append and popleft)
  queue = collections.deque()

  # Start the traversal
  visited.add(start_node)
  queue.append(start_node)

  # List to store the order of visited nodes
  traversal_order = []

  while queue: # While there are nodes to process
      # Dequeue a vertex from the front of the queue
      current_node = queue.popleft()
      traversal_order.append(current_node)
      # print(f"Visiting node: {current_node}") # Optional: print node being visited

      # Get all adjacent vertices of the dequeued vertex
      # Use graph.get(current_node, []) to handle nodes that might be listed
      # as neighbors but don't have their own entry in the graph dictionary
      # (e.g., nodes with no outgoing edges).
      for neighbor in graph.get(current_node, []):
          if neighbor not in visited:
              visited.add(neighbor)
              queue.append(neighbor)
  
  
  print(f"Elapsed time: {time.perf_counter() - start_time:.20f} seconds")
  return traversal_order

print(bfs(graph_bfs_dfs, 2))

Elapsed time: 0.00001940000220201910 seconds
[2, 0, 3, 1]


### DFS (depth first search)

In [307]:
def dfs_recursive_helper(graph, node, visited):
  """
  Helper function for recursive DFS.

  Args:
    graph: The graph (adjacency list).
    node: The current node to visit.
    visited: A set keeping track of visited nodes.
  """

  if node not in visited:
    visited.add(node)
    print(node, end=' ') # Process the node (e.g., print it)

    # Recur for all adjacent vertices
    # Use graph.get(node, []) to handle nodes with no outgoing edges gracefully
    for neighbor in graph.get(node, []):
      if neighbor not in visited:
        dfs_recursive_helper(graph, neighbor, visited)

def dfs_recursive(graph, start_node):
  """
  Performs Depth First Search starting from a given node using recursion.

  Args:
    graph: The graph (adjacency list).
    start_node: The node to start the traversal from.
  """
  start_time = time.perf_counter()

  visited = set() # Keep track of visited nodes
  print(f"DFS Recursive Traversal starting from {start_node}:")
  if start_node not in graph:
      print(f"  Start node '{start_node}' not found in the graph.")
      return

  dfs_recursive_helper(graph, start_node, visited)
  print(f"\nElapsed time: {time.perf_counter() - start_time:.20f} seconds")
  print("\nTraversal finished.")

print(dfs_recursive(graph_bfs_dfs, 0))

def dfs_iterative(graph, start_node):
  """
  Performs Depth First Search starting from a given node using iteration and a stack.

  Args:
    graph: The graph (adjacency list).
    start_node: The node to start the traversal from.
  """
  start_time = time.perf_counter()

  print(f"DFS Iterative Traversal starting from {start_node}:")
  if start_node not in graph:
      print(f"  Start node '{start_node}' not found in the graph.")
      return

  visited = set()      # Keep track of visited nodes
  # stack = collections.deque([start_node]) # More efficient deque
  stack = [start_node] # Using a list as a stack

  while stack:
    node = stack.pop() # Get the last node added (LIFO - Last In, First Out)

    if node not in visited:
      visited.add(node)
      print(node, end=' ') # Process the node

      # Add unvisited neighbors to the stack.
      # Iterate in reverse to roughly mimic the recursive exploration order
      # (explores the 'first' neighbor in the list first).
      # Use graph.get(node, []) to handle nodes with no outgoing edges.
      neighbors = graph.get(node, [])
      # Use reversed() if you want to visit neighbors in the order they appear
      # in the adjacency list (similar to the typical recursive call order).
      # If the specific visit order among siblings doesn't matter, you can omit reversed().
      for neighbor in reversed(neighbors):
        if neighbor not in visited:
          stack.append(neighbor)

  end_time = time.perf_counter()
  elapsed_time = end_time - start_time
  print(f"Elapsed time: {elapsed_time:.20f} seconds")
  print("\nTraversal finished.")

print(dfs_iterative(graph_bfs_dfs, 2))

DFS Recursive Traversal starting from 0:
0 1 2 3 
Elapsed time: 0.00091919999977108091 seconds

Traversal finished.
None
DFS Iterative Traversal starting from 2:
2 0 1 3 Elapsed time: 0.00016839999807416461 seconds

Traversal finished.
None


## Kruskal's

Below is the graph used to test Kruskal's 

In [308]:
edges = [
        (10, 0, 1),
        (6, 0, 2),
        (5, 0, 3),
        (15, 1, 3),
        (4, 2, 3)
    ]

In [309]:
class DSU:
    def __init__(self, num_vertices):
        # Initialize parent array: each node is its own parent initially
        # Initialize rank array: used for union by rank optimization
        self.parent = list(range(num_vertices))
        self.rank = [0] * num_vertices

    # Find the representative (root) of the set containing element i
    # Implements path compression optimization
    def find(self, i):
        if self.parent[i] == i:
            return i
        # Path compression: make nodes point directly to the root
        self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    # Union (merge) the sets containing elements x and y
    # Implements union by rank optimization
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)

        if root_x != root_y:  # Only union if they are in different sets
            # Attach the smaller rank tree under the root of the higher rank tree
            if self.rank[root_x] < self.rank[root_y]:
                self.parent[root_x] = root_y
            elif self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            else:
                # If ranks are same, make one root and increment its rank
                self.parent[root_y] = root_x
                self.rank[root_x] += 1
            return True # Union was performed
        return False # x and y were already in the same set


# Kruskal's algorithm implementation

def kruskal_mst(graph_edges, num_vertices):
    """
    Finds the Minimum Spanning Tree (MST) of a graph using Kruskal's algorithm.

    Args:
        graph_edges (list): A list of tuples representing edges.
                            Each tuple should be (weight, u, v), where 'u' and 'v'
                            are the vertices connected by the edge (0-indexed)
                            and 'weight' is the edge weight.
        num_vertices (int): The total number of vertices in the graph.

    Returns:
        tuple: A tuple containing:
               - mst (list): A list of edges (u, v, weight) forming the MST.
               - mst_weight (int or float): The total weight of the MST.
               Returns ([], 0) if the graph has no vertices or edges.
               If the graph is disconnected, it returns a Minimum Spanning Forest (MSF).
    """
    start_time = time.perf_counter()

    if num_vertices <= 0:
        return [], 0

    # 1. Sort all edges by weight in ascending order
    #    Using lambda to sort based on the first element (weight)
    sorted_edges = sorted(graph_edges, key=lambda item: item[0])

    mst = []              # Stores the edges of the MST
    mst_weight = 0        # Stores the total weight of the MST
    edges_in_mst = 0      # Counter for edges added to MST

    dsu = DSU(num_vertices) # Initialize DSU structure

    # 2. Iterate through sorted edges
    for weight, u, v in sorted_edges:
        # 3. Check if adding the edge (u, v) creates a cycle
        #    This is done by checking if u and v are already in the same set
        if dsu.union(u, v): # If union returns True, no cycle was formed
            # 4. If no cycle, add the edge to the MST
            mst.append((u, v, weight))
            mst_weight += weight
            edges_in_mst += 1

            # 5. Optimization: Stop if we have V-1 edges (MST is complete for connected graphs)
            if edges_in_mst == num_vertices - 1:
                break

    # Note: If edges_in_mst < num_vertices - 1 after the loop,
    # the original graph was disconnected, and we have found a Minimum Spanning Forest.
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.20f} seconds")
    return mst, mst_weight


print(kruskal_mst(edges, 4))

Elapsed time: 0.00004900000203633681 seconds
([(2, 3, 4), (0, 3, 5), (0, 1, 10)], 19)


## Prim's

Below is the graph used to test Prim's algorithm:

In [310]:
prim_graph = {
    0: [(1, 10), (2, 6), (3, 5)],
    1: [(0, 10), (3, 15)],
    2: [(0, 6), (3, 4)],
    3: [(0, 5), (1, 15), (2, 4)]
}

In [311]:
def prim_mst(graph, start_node):
    start_time = time.perf_counter()
    """
    Calculates the Minimum Spanning Tree (MST) of a connected, undirected,
    weighted graph using Prim's algorithm.

    Args:
        graph (dict): A representation of the graph as an adjacency list.
                      Format: {
                          'vertex1': [('neighbor1', weight1), ('neighbor2', weight2), ...],
                          'vertex2': [...],
                          ...
                      }
        start_node: The vertex to start building the MST from.

    Returns:
        tuple: A tuple containing:
            - set: A set of tuples representing the edges in the MST.
                   Each tuple is (u, v, weight), where u and v are vertices
                   and weight is the edge weight. Edges are stored canonically
                   (smaller vertex first) to avoid duplicates like (A, B, w)
                   and (B, A, w).
            - float: The total weight of the MST.
        Returns (None, 0) if the graph is empty or the start_node is invalid.
        Raises ValueError if the start_node is not in the graph.

    Note:
        - Assumes the input graph is connected. If disconnected, it will only
          find the MST for the connected component containing the start_node.
        - Assumes edge weights are non-negative.
    """
    if not graph:
        print("Error: Graph is empty.")
        return set(), 0.0

    if start_node not in graph:
        raise ValueError(f"Start node '{start_node}' not found in the graph.")

    mst_edges = set()            # Set to store edges (u, v, weight) in the MST
    visited = set()            # Set to keep track of vertices included in the MST
    min_heap = []              # Priority queue (min-heap) to store potential edges
                               # Format: (weight, from_vertex, to_vertex)
    total_weight = 0.0

    # --- Initialization ---
    # Add the starting node to the visited set
    visited.add(start_node)

    # Add all edges connected to the start_node to the min_heap
    for neighbor, weight in graph.get(start_node, []):
        heapq.heappush(min_heap, (weight, start_node, neighbor))

    # --- Main Loop ---
    # Continue while the priority queue is not empty AND
    # we haven't visited all nodes (needed for connected graph check)
    # (In a guaranteed connected graph, len(visited) < len(graph) would suffice)
    while min_heap and len(visited) < len(graph):
        # 1. Extract the edge with the minimum weight
        weight, u, v = heapq.heappop(min_heap)

        # 2. Check if the destination vertex 'v' is already visited
        if v in visited:
            continue # This edge connects two vertices already in MST, skip it

        # 3. Add the vertex 'v' and the edge (u, v) to the MST
        visited.add(v)
        # Store edge canonically (smaller vertex first) for consistency
        edge = tuple(sorted((u, v))) + (weight,)
        mst_edges.add(edge)
        total_weight += weight

        # 4. Add new potential edges from the newly added vertex 'v'
        #    to its neighbors that are *not* yet in the MST
        for neighbor, edge_weight in graph.get(v, []):
            if neighbor not in visited:
                heapq.heappush(min_heap, (edge_weight, v, neighbor))

    # --- Check for disconnected graph ---
    if len(visited) != len(graph):
        print(f"Warning: Graph might be disconnected. MST found only for the "
              f"component containing '{start_node}'. Visited {len(visited)} out "
              f"of {len(graph)} vertices.")

    print(f"Elapsed time: {time.perf_counter() - start_time:.20f} seconds")
    return mst_edges, total_weight

prim_graph = {
    0: [(1, 10), (2, 6), (3, 5)],
    1: [(0, 10), (3, 15)],
    2: [(0, 6), (3, 4)],
    3: [(0, 5), (1, 15), (2, 4)]
}

print(prim_mst(prim_graph, 0))

Elapsed time: 0.00004260000059730373 seconds
({(0, 3, 5), (0, 1, 10), (2, 3, 4)}, 19.0)


# Dijkstra’s and Bellman-Ford

Below is the graph we used to test Dijkstra’s algorithm:

In [312]:
edges_shortest_path = {
    0: [(1, 4), (2, 2)],
    1: [(0, 4), (2, 5), (3, 10)],
    2: [(0, 2), (1, 5), (3, 3), (4, 2)],
    3: [(1, 10), (2, 3), (4, 4), (5, 11)],
    4: [(2, 2), (3, 4), (5, 8)],
    5: [(3, 11), (4, 8)]
}

### Dijkstra's

In [313]:
def dijkstra(graph, start_node):
    start_time = time.perf_counter()

    """
    Implements Dijkstra's shortest path algorithm.

    Args:
        graph (dict): A dictionary representing the graph where keys are nodes
                      and values are lists of tuples (neighbor, weight).
                      Example: {'A': [('B', 1), ('C', 4)], 'B': [('A', 1), ...]}
        start_node: The node from which to calculate shortest paths.

    Returns:
        tuple: A tuple containing:
            - distances (dict): A dictionary mapping each reachable node to its
                                shortest distance from the start_node.
            - predecessors (dict): A dictionary mapping each node (except start)
                                    to its predecessor node on the shortest path
                                    from the start_node. Useful for reconstructing
                                    the path.
    Raises:
        ValueError: If start_node is not in the graph.
        TypeError: If edge weights are not numeric or are negative.
    """

    # --- Input Validation ---
    if start_node not in graph:
        raise ValueError(f"Start node '{start_node}' not found in the graph.")

    # Validate weights (optional but good practice)
    for node, edges in graph.items():
        for neighbor, weight in edges:
            if not isinstance(weight, (int, float)):
                raise TypeError(f"Edge weight ({weight}) from {node} to {neighbor} is not numeric.")
            if weight < 0:
                raise ValueError("Dijkstra's algorithm does not support negative edge weights.")


    # --- Initialization ---
    # Distances: Store the shortest distance found *so far* from start_node to every other node.
    # Initialize all distances to infinity, except for the start_node (distance 0).
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0

    # Predecessors: Store the preceding node in the shortest path found so far.
    # Useful for reconstructing the path later.
    predecessors = {node: None for node in graph}

    # Priority Queue: Stores tuples of (current_distance, node).
    # Python's heapq is a min-heap, so it will always give us the node
    # with the smallest distance among the unvisited nodes.
    # We start with the source node at distance 0.
    priority_queue = [(0, start_node)] # (distance, node)

    # --- Main Loop ---
    while priority_queue:
        # Get the node with the smallest distance from the priority queue
        current_distance, current_node = heapq.heappop(priority_queue)

        # Optimization: If we've already found a *shorter* path to current_node
        # than the one stored in this heap entry, skip this entry. This handles
        # cases where we added multiple entries for the same node to the heap
        # before processing the shortest one.
        if current_distance > distances[current_node]:
            continue

        # --- Explore Neighbors ---
        # Iterate through neighbors of the current node
        for neighbor, weight in graph.get(current_node, []): # Use .get for safety if node might have no outgoing edges listed
            distance_through_current = current_distance + weight

            # --- Relaxation Step ---
            # If we found a shorter path to the neighbor through the current_node:
            if distance_through_current < distances[neighbor]:
                # Update the shortest distance to the neighbor
                distances[neighbor] = distance_through_current
                # Record the predecessor for path reconstruction
                predecessors[neighbor] = current_node
                # Add the neighbor to the priority queue with its new, shorter distance
                heapq.heappush(priority_queue, (distance_through_current, neighbor))

    # --- Return Results ---
    # Remove nodes that are unreachable (still have infinite distance) from the final dicts
    # Or keep them to explicitly show they are unreachable
    # reachable_distances = {k: v for k, v in distances.items() if v != float('inf')}
    # reachable_predecessors = {k: v for k, v in predecessors.items() if distances[k] != float('inf')}
    print(f"Elapsed time: {time.perf_counter() - start_time:.20f} seconds")

    return distances, predecessors

def reconstruct_path(predecessors, start_node, end_node):
    """
    Reconstructs the shortest path from start_node to end_node using the
    predecessor information generated by Dijkstra's algorithm.

    Args:
        predecessors (dict): The dictionary returned by the dijkstra function.
        start_node: The starting node of the path.
        end_node: The ending node of the path.

    Returns:
        list: A list of nodes representing the shortest path from start_node
              to end_node. Returns None if no path exists. Returns [start_node]
              if start_node == end_node. Returns None if end_node is not reachable.
    """
    if end_node not in predecessors:
         # This case handles if end_node wasn't even in the graph keys originally
         print(f"Warning: End node '{end_node}' was not part of the graph processed.")
         return None
    if predecessors.get(end_node) is None and start_node != end_node:
         # This handles nodes that were in the graph but unreachable from start_node
         return None # No path exists or end_node is unreachable

    path = collections.deque()
    current_node = end_node
    while current_node is not None:
        path.appendleft(current_node)
        if current_node == start_node:
            break
        current_node = predecessors.get(current_node) # Use .get in case of issues

        # Safety check for potential cycles if predecessors dict was malformed
        # or if start_node is somehow unreachable from end_node via predecessors
        if current_node is None and path[0] != start_node:
            print(f"Error reconstructing path. Could not trace back to start node '{start_node}' from '{end_node}'.")
            return None

    return list(path)

print(dijkstra(edges_shortest_path, 0))

Elapsed time: 0.00005240000245976262 seconds
({0: 0, 1: 4, 2: 2, 3: 5, 4: 4, 5: 12}, {0: None, 1: 0, 2: 0, 3: 2, 4: 2, 5: 4})


### Bellman's

Below is the graph used for Bellman's Algorithm:

In [314]:
edges_bellman = [
        (0, 1, -1),
        (0, 2, 4),
        (1,  2, 3),
        (1, 3, 2),
        (1, 4, 2),
        (2, 2, 5),
        (2, 1, 1),
        (4, 3, -3)
    ]
    

In [315]:
INF = float('inf')

def bellman_ford(graph, num_vertices, source):
    start_time = time.perf_counter()

    """
    Implements the Bellman-Ford algorithm to find the shortest paths
    from a source vertex to all other vertices in a weighted graph.

    Args:
        graph (list): A list of edges, where each edge is a tuple
                      (u, v, weight) representing an edge from vertex u
                      to vertex v with the given weight. Vertex labels
                      are assumed to be integers from 0 to num_vertices - 1.
        num_vertices (int): The total number of vertices in the graph.
        source (int): The starting vertex (0-based index).

    Returns:
        tuple: A tuple containing:
            - distances (dict): A dictionary mapping each vertex (int) to its
                                shortest distance from the source. Unreachable
                                vertices will have a distance of INF.
            - predecessors (dict): A dictionary mapping each vertex (int) to its
                                   predecessor node on the shortest path from
                                   the source. The source and unreachable vertices
                                   will have None as their predecessor.
            Returns (None, None) if a negative cycle reachable from the source
            is detected.

    Raises:
        ValueError: If the source vertex index is invalid.
        ValueError: If num_vertices is non-positive.
    """

    if not 0 <= source < num_vertices:
        raise ValueError(f"Source vertex {source} is out of range [0, {num_vertices-1}]")
    if num_vertices <= 0:
        raise ValueError("Number of vertices must be positive.")

    # 1. Initialize distances and predecessors
    distances = {i: INF for i in range(num_vertices)}
    predecessors = {i: None for i in range(num_vertices)}
    distances[source] = 0

    # 2. Relax edges repeatedly (V-1 times)
    # For a graph with V vertices, V-1 iterations are sufficient if there are
    # no negative cycles, as the longest possible simple path has V-1 edges.
    for _ in range(num_vertices - 1):
        updated = False # Optimization: stop early if no updates occur in an iteration
        for u, v, weight in graph:
            # Check if a shorter path to v is found through u
            if distances[u] != INF and distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                predecessors[v] = u
                updated = True
        if not updated: # If no distances were updated, shortest paths are found
            break

    # 3. Check for negative-weight cycles
    # If we can still relax an edge after V-1 iterations, it means there's a
    # negative cycle reachable from the source.
    for u, v, weight in graph:
        if distances[u] != INF and distances[u] + weight < distances[v]:
            print(f"Graph contains a negative weight cycle reachable from source {source}!")
            print(f"Edge ({u}, {v}) with weight {weight} can still be relaxed.")
            # Optionally: identify and return the cycle itself (more complex)
            return None, None # Indicate negative cycle

    # 4. Return results
    print("*** Note: replace numbers with their letter counterparts (ex. A = 0, B = 1, etc).")
    print(f"Elapsed time: {time.perf_counter() - start_time:.20f} seconds")

    return distances, predecessors

def reconstruct_path(predecessors, source, target):
    """
    Reconstructs the shortest path from source to target using the predecessors dict.

    Args:
        predecessors (dict): The predecessors dictionary returned by bellman_ford.
        source (int): The source vertex.
        target (int): The target vertex.

    Returns:
        list: A list of vertices representing the path from source to target,
              or None if no path exists.
    """
    if predecessors is None or target not in predecessors: # Handle negative cycle case or invalid target
         return None
    if predecessors.get(target) is None and target != source: # Target is unreachable
        return None

    path = []
    current = target
    while current is not None:
        path.append(current)
        # Check for cycle in reconstruction (shouldn't happen if Bellman-Ford finished without error)
        if current == predecessors[current]:
             print(f"Error: Cycle detected during path reconstruction at node {current}")
             return None # Or raise an error
        current = predecessors[current]
        # Safety break to prevent infinite loops in unexpected scenarios
        if len(path) > len(predecessors) * 2:
             print(f"Error: Path reconstruction seems to be looping.")
             return None # Or raise an error

    # The path is built backwards, so reverse it
    path.reverse()

    # Check if the path actually starts at the source
    if path[0] == source:
        return path
    else:
        # This means the target was reachable, but not from the specified source
        # (might happen in disconnected graphs or if source couldn't reach target)
        return None
    
print(bellman_ford(edges_bellman, 5, 0))

*** Note: replace numbers with their letter counterparts (ex. A = 0, B = 1, etc).
Elapsed time: 0.00026450000223121606 seconds
({0: 0, 1: -1, 2: 2, 3: -2, 4: 1}, {0: None, 1: 0, 2: 1, 3: 4, 4: 1})
